### Drive Data

In [1]:

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 3.4MB/s 


In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls '/content/drive/My Drive/AV_data/Quantum_Black_5th_July_2019/'

drive_csv_file.zip	 trip_csv_file.zip  weather_csv_file.zip
drive_vehcile_df_v1.csv  vehicle.csv


In [0]:
# !unzip -q '/content/drive/My Drive/AV_data/input_data.zip'

### Load Packages

In [0]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import os
from pathlib import Path
import datetime
from datetime import timedelta

In [0]:
drive_df = pd.read_csv('/content/drive/My Drive/AV_data/Quantum_Black_5th_July_2019/drive_csv_file.zip')

In [7]:
drive_df.head(3)

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,iat,rpm
0,1000517,e117220c8b15455f800a4b5b0c4c52a7,2017-01-27 12:00:00,0.00,48.0,76.22,78.92,97.0,223.21,170.91,88.0,2066.48
1,1000517,e117220c8b15455f800a4b5b0c4c52a7,2017-01-27 12:00:01,44.52,55.0,75.44,83.57,93.0,211.91,167.83,75.0,2057.53
2,1000517,e117220c8b15455f800a4b5b0c4c52a7,2017-01-27 12:00:02,60.93,49.0,68.18,83.03,99.0,224.20,177.68,94.0,2046.12


In [8]:
drive_df.shape

(9217528, 12)

In [9]:
min(drive_df['velocity'])

-12.7

In [0]:
drive_df_trips = drive_df[['trip_id', 'datetime', 'velocity']]

In [0]:
from datetime import datetime
from pytz import timezone


def convert_date_timezone(x):  
  datetime_obj_naive = datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S')
  # Right way!
  datetime_obj_pacific = datetime_obj_naive.astimezone(timezone('US/Pacific'))
  return (datetime_obj_pacific.strftime("%Y-%m-%d %H:%M:%S"))

In [12]:
drive_df_trips['datetime'] =  drive_df_trips['datetime'].apply(lambda x :convert_date_timezone(x)) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
drive_df_trips = drive_df_trips.sort_values(['trip_id', 'datetime'], ascending=[True, True])

In [14]:
drive_df_trips.tail(5)

,trip_id,datetime,velocity
2086189,ffe2c04ae0de450cae6f41c7e2fbd86d,2017-01-21 04:05:34,59.06
2086190,ffe2c04ae0de450cae6f41c7e2fbd86d,2017-01-21 04:05:35,59.04
2086191,ffe2c04ae0de450cae6f41c7e2fbd86d,2017-01-21 04:05:36,54.83
2086192,ffe2c04ae0de450cae6f41c7e2fbd86d,2017-01-21 04:05:37,20.00
2086193,ffe2c04ae0de450cae6f41c7e2fbd86d,2017-01-21 04:05:38,0.00


In [15]:
drive_df_trips.dtypes

trip_id      object
datetime     object
velocity    float64
dtype: object

In [0]:
# lag datetime
drive_df_trips['datetime_1'] = drive_df_trips['datetime']

In [17]:
drive_df_trips.head(4)

,trip_id,datetime,velocity,datetime_1
8292551,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:00,0.00,2017-01-06 15:00:00
8292552,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:01,61.36,2017-01-06 15:00:01
8292553,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:02,47.50,2017-01-06 15:00:02
8292554,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:03,48.28,2017-01-06 15:00:03


In [18]:
drive_df_trips.isnull().sum()

trip_id       0
datetime      0
velocity      0
datetime_1    0
dtype: int64

In [0]:
grouped_df = drive_df_trips.groupby(["trip_id"])

In [0]:
def lag_by_group(key, value_df):
    df = value_df.assign(group = key) # this pandas method returns a copy of the df, with group columns assigned the key value
    return (df.sort_values(by=["datetime", "trip_id"], ascending=True)
        .set_index(["datetime", "trip_id"])
        .shift(1)
               ) 

In [0]:
dflist = [lag_by_group(g, grouped_df.get_group(g)) for g in grouped_df.groups.keys()]

dflist_1 = pd.concat(dflist, axis=0).reset_index()

In [0]:
dflist_1.drop(['group'],axis=1,inplace=True)

In [24]:
dflist_1.shape

(9217528, 4)

In [25]:
dflist_1.head(3)

,datetime,trip_id,velocity,datetime_1
0,2017-01-06 15:00:00,00922df3be5a4589ab385d0c2da2dd81,NaN,NaN
1,2017-01-06 15:00:01,00922df3be5a4589ab385d0c2da2dd81,0.00,2017-01-06 15:00:00
2,2017-01-06 15:00:02,00922df3be5a4589ab385d0c2da2dd81,61.36,2017-01-06 15:00:01


In [0]:
dflist_1.rename(columns = {'velocity':'lag_velocity'}, inplace = True)
dflist_1.rename(columns = {'datetime_1':'lag_datetime'}, inplace = True)

In [27]:
dflist_1.head(3)

,datetime,trip_id,lag_velocity,lag_datetime
0,2017-01-06 15:00:00,00922df3be5a4589ab385d0c2da2dd81,NaN,NaN
1,2017-01-06 15:00:01,00922df3be5a4589ab385d0c2da2dd81,0.00,2017-01-06 15:00:00
2,2017-01-06 15:00:02,00922df3be5a4589ab385d0c2da2dd81,61.36,2017-01-06 15:00:01


In [0]:
df_new = pd.merge(drive_df_trips, dflist_1,  how='left', left_on=['datetime','trip_id'], right_on = ['datetime','trip_id'])

In [29]:
df_new.shape

(9217528, 6)

In [30]:
df_new.head(3)

,trip_id,datetime,velocity,datetime_1,lag_velocity,lag_datetime
0,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:00,0.00,2017-01-06 15:00:00,NaN,NaN
1,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:01,61.36,2017-01-06 15:00:01,0.00,2017-01-06 15:00:00
2,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:02,47.50,2017-01-06 15:00:02,61.36,2017-01-06 15:00:01


In [31]:
df_new['trip_id'].unique()

array(['00922df3be5a4589ab385d0c2da2dd81',
       '00dc31fe55e24d14989c89de4b3b683b',
       '0156d21e316d4d8b9d5bf6ccff797bf7', ...,
       'ffcdf94f51a54c85bb6d90b3b8624f6b',
       'ffda453e86e34db587018c3333f549a0',
       'ffe2c04ae0de450cae6f41c7e2fbd86d'], dtype=object)

In [0]:
df_new = df_new.dropna()

In [0]:
drive_df_trips = df_new.copy()
del df_new
del dflist_1
del grouped_df

In [0]:
## COnvert the velocity into m/s
drive_df_trips['velocity'] = drive_df_trips['velocity'] * (5.0/18.0)
drive_df_trips['lag_velocity'] = drive_df_trips['lag_velocity'] * (5.0/18.0)

In [35]:
drive_df_trips.head(3)

,trip_id,datetime,velocity,datetime_1,lag_velocity,lag_datetime
1,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:01,17.044444,2017-01-06 15:00:01,0.000000,2017-01-06 15:00:00
2,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:02,13.194444,2017-01-06 15:00:02,17.044444,2017-01-06 15:00:01
3,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:03,13.411111,2017-01-06 15:00:03,13.194444,2017-01-06 15:00:02


#### Create lagged variables to - Calculate Acceleration

In [36]:
drive_df_trips.dtypes

trip_id          object
datetime         object
velocity        float64
datetime_1       object
lag_velocity    float64
lag_datetime     object
dtype: object

In [37]:
drive_df_trips.head(3)

,trip_id,datetime,velocity,datetime_1,lag_velocity,lag_datetime
1,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:01,17.044444,2017-01-06 15:00:01,0.000000,2017-01-06 15:00:00
2,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:02,13.194444,2017-01-06 15:00:02,17.044444,2017-01-06 15:00:01
3,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:03,13.411111,2017-01-06 15:00:03,13.194444,2017-01-06 15:00:02


In [38]:
drive_df_trips.isnull().sum()

trip_id         0
datetime        0
velocity        0
datetime_1      0
lag_velocity    0
lag_datetime    0
dtype: int64

In [0]:
# ## getthe lag and fill NA values
# drive_df_trips['lag_datetime'] = drive_df_trips['datetime'].shift(1)



# # Get the difference in velocities
# drive_df_trips['lag_velocity'] = drive_df_trips['velocity'].shift(1)

# # drive_df_trips = drive_df_trips.dropna()

drive_df_trips['velocity_diff'] = drive_df_trips['velocity'] - drive_df_trips['lag_velocity']


In [40]:
drive_df_trips.head(3)

,trip_id,datetime,velocity,datetime_1,lag_velocity,lag_datetime,velocity_diff
1,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:01,17.044444,2017-01-06 15:00:01,0.000000,2017-01-06 15:00:00,17.044444
2,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:02,13.194444,2017-01-06 15:00:02,17.044444,2017-01-06 15:00:01,-3.850000
3,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:03,13.411111,2017-01-06 15:00:03,13.194444,2017-01-06 15:00:02,0.216667


In [41]:
drive_df_trips.isnull().sum()

trip_id          0
datetime         0
velocity         0
datetime_1       0
lag_velocity     0
lag_datetime     0
velocity_diff    0
dtype: int64

In [42]:
drive_df_trips

,trip_id,datetime,velocity,datetime_1,lag_velocity,lag_datetime,velocity_diff
1,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:01,17.044444,2017-01-06 15:00:01,0.000000,2017-01-06 15:00:00,17.044444
2,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:02,13.194444,2017-01-06 15:00:02,17.044444,2017-01-06 15:00:01,-3.850000
3,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:03,13.411111,2017-01-06 15:00:03,13.194444,2017-01-06 15:00:02,0.216667
4,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:04,13.361111,2017-01-06 15:00:04,13.411111,2017-01-06 15:00:03,-0.050000
5,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:05,14.805556,2017-01-06 15:00:05,13.361111,2017-01-06 15:00:04,1.444444
6,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:06,14.452778,2017-01-06 15:00:06,14.805556,2017-01-06 15:00:05,-0.352778
7,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:07,14.777778,2017-01-06 15:00:07,14.452778,2017-01-06 15:00:06,0.325000
8,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:08,13.305556,2017-01-06 15:00:08,14.777778,2017-01-06 15:00:07,-1.472222
9,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:09,13.702778,2017-01-06 15:00:09,13.305556,2017-01-06 15:00:08,0.397222
10,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:10,16.133333,2017-01-06 15:00:10,13.702778,2017-01-06 15:00:09,2.430556


In [0]:
from datetime import datetime, time
def convert_datetime(x): 
  return datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S')  

In [0]:
drive_df_trips['time_diff'] =  drive_df_trips['datetime'].apply(lambda x :convert_datetime(x)) - drive_df_trips['lag_datetime'].apply(lambda x: convert_datetime(x)) 

In [45]:
drive_df_trips.dtypes

trip_id                   object
datetime                  object
velocity                 float64
datetime_1                object
lag_velocity             float64
lag_datetime              object
velocity_diff            float64
time_diff        timedelta64[ns]
dtype: object

In [46]:
drive_df_trips.head(5)

,trip_id,datetime,velocity,datetime_1,lag_velocity,lag_datetime,velocity_diff,time_diff
1,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:01,17.044444,2017-01-06 15:00:01,0.000000,2017-01-06 15:00:00,17.044444,00:00:01
2,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:02,13.194444,2017-01-06 15:00:02,17.044444,2017-01-06 15:00:01,-3.850000,00:00:01
3,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:03,13.411111,2017-01-06 15:00:03,13.194444,2017-01-06 15:00:02,0.216667,00:00:01
4,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:04,13.361111,2017-01-06 15:00:04,13.411111,2017-01-06 15:00:03,-0.050000,00:00:01
5,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:05,14.805556,2017-01-06 15:00:05,13.361111,2017-01-06 15:00:04,1.444444,00:00:01


In [47]:
drive_df_trips[drive_df_trips['velocity'] == 0].shape

(1708, 8)

In [0]:
def convert_to_seconds(x):
  return x.days * 24 * 3600 + x.seconds 
  

In [0]:
drive_df_trips['time_diff_secs'] = drive_df_trips['time_diff'].apply(lambda x:convert_to_seconds(x))

In [50]:
drive_df_trips.head(3)

,trip_id,datetime,velocity,datetime_1,lag_velocity,lag_datetime,velocity_diff,time_diff,time_diff_secs
1,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:01,17.044444,2017-01-06 15:00:01,0.000000,2017-01-06 15:00:00,17.044444,00:00:01,1
2,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:02,13.194444,2017-01-06 15:00:02,17.044444,2017-01-06 15:00:01,-3.850000,00:00:01,1
3,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:03,13.411111,2017-01-06 15:00:03,13.194444,2017-01-06 15:00:02,0.216667,00:00:01,1


In [51]:
drive_df_trips[(drive_df_trips['time_diff_secs'] >= 2) & (drive_df_trips['time_diff_secs'] < 0)].shape

(0, 9)

In [0]:
drive_df_trips = drive_df_trips[(drive_df_trips['time_diff_secs'] < 2) & (drive_df_trips['time_diff_secs'] >= 0)]

In [53]:
drive_df_trips.head(3)

,trip_id,datetime,velocity,datetime_1,lag_velocity,lag_datetime,velocity_diff,time_diff,time_diff_secs
1,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:01,17.044444,2017-01-06 15:00:01,0.000000,2017-01-06 15:00:00,17.044444,00:00:01,1
2,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:02,13.194444,2017-01-06 15:00:02,17.044444,2017-01-06 15:00:01,-3.850000,00:00:01,1
3,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:03,13.411111,2017-01-06 15:00:03,13.194444,2017-01-06 15:00:02,0.216667,00:00:01,1


In [54]:
drive_df_trips['time_diff_secs'].describe()

count    9215820.0
mean           1.0
std            0.0
min            1.0
25%            1.0
50%            1.0
75%            1.0
max            1.0
Name: time_diff_secs, dtype: float64

In [0]:
drive_df_trips['acceleration'] = drive_df_trips['velocity_diff'] / drive_df_trips['time_diff_secs']

In [56]:
drive_df_trips.head(3)

,trip_id,datetime,velocity,datetime_1,lag_velocity,lag_datetime,velocity_diff,time_diff,time_diff_secs,acceleration
1,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:01,17.044444,2017-01-06 15:00:01,0.000000,2017-01-06 15:00:00,17.044444,00:00:01,1,17.044444
2,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:02,13.194444,2017-01-06 15:00:02,17.044444,2017-01-06 15:00:01,-3.850000,00:00:01,1,-3.850000
3,00922df3be5a4589ab385d0c2da2dd81,2017-01-06 15:00:03,13.411111,2017-01-06 15:00:03,13.194444,2017-01-06 15:00:02,0.216667,00:00:01,1,0.216667


In [57]:
drive_df_trips.isnull().sum()

trip_id           0
datetime          0
velocity          0
datetime_1        0
lag_velocity      0
lag_datetime      0
velocity_diff     0
time_diff         0
time_diff_secs    0
acceleration      0
dtype: int64

In [0]:
#replace with NA 0
drive_df_trips['acceleration'].fillna(0, inplace = True)

In [0]:
def convert_datetime_list(datetimelist): 
  return [datetime.datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S') for x in datetimelist]  

In [0]:
drive_df_trips = drive_df_trips.sort_values(['trip_id', 'datetime'], ascending=[True, True])

In [0]:
drive_df_trips.drop(['velocity','lag_datetime', 'lag_velocity', 'velocity_diff', 'time_diff'],axis=1,inplace=True)

In [0]:
# Transform the data into lists for each of the columns with an unique index column as trip_id
drive_df_trips_lst = drive_df_trips.groupby('trip_id').agg(pd.Series.tolist).reset_index()

In [63]:
drive_df_trips_lst.head(2)

,trip_id,datetime,datetime_1,time_diff_secs,acceleration
0,00922df3be5a4589ab385d0c2da2dd81,"[2017-01-06 15:00:01, 2017-01-06 15:00:02, 201...","[2017-01-06 15:00:01, 2017-01-06 15:00:02, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[17.044444444444444, -3.8499999999999996, 0.21..."
1,00dc31fe55e24d14989c89de4b3b683b,"[2017-01-20 17:00:01, 2017-01-20 17:00:02, 201...","[2017-01-20 17:00:01, 2017-01-20 17:00:02, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[19.927777777777777, 2.052777777777777, -0.216..."


In [64]:
drive_df_trips_lst.shape

(1708, 5)

### Fields Preparation 

#### ft_cnt_vehicle_deaccel_val

In [0]:
def get_ft_cnt_vehicle_deaccel_val(lst, datetimelist):  

  datetimelist_updated = [datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S') for x in datetimelist]

  new_list = []  
  count = 0
  flag = 0
  i = 0
  nStartIndexPerBlock = 0
  nEndIndexPerBlock   = 0
  nDiffSumofTimes = 0
  n = len(lst)

  for i in range(0, len(lst)):
    if (i+1 < len(lst)):  
      if (lst[i] >= lst[i+1]):
        if(flag != 1):
          nStartIndexPerBlock = i
        flag = 1
        new_list.append(0)
        continue
      elif (lst[i] < lst[i+1]):       
        if(flag == 1):
          count = count+1
          new_list.append(1)
          nEndIndexPerBlock = i
          nDiffSumofTimes = nDiffSumofTimes + int((datetimelist_updated[nEndIndexPerBlock] - datetimelist_updated[nStartIndexPerBlock]).total_seconds())
          flag = 0
        else:        
          new_list.append(0)
        continue
  if(flag == 1):  
    new_list.append(1)
    count = count + 1

    nDiffSumofTimes = nDiffSumofTimes + int((datetimelist_updated[n-1] - datetimelist_updated[nStartIndexPerBlock]).total_seconds())
    final_seconds = nDiffSumofTimes 
  else:
    new_list.append(0)
  return count
	

In [0]:
drive_df_trips_lst['ft_cnt_vehicle_deaccel_val']  = drive_df_trips_lst.apply(lambda row: get_ft_cnt_vehicle_deaccel_val(row['acceleration'], row['datetime']), axis=1)

In [67]:
drive_df_trips_lst.head(2)

,trip_id,datetime,datetime_1,time_diff_secs,acceleration,ft_cnt_vehicle_deaccel_val
0,00922df3be5a4589ab385d0c2da2dd81,"[2017-01-06 15:00:01, 2017-01-06 15:00:02, 201...","[2017-01-06 15:00:01, 2017-01-06 15:00:02, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[17.044444444444444, -3.8499999999999996, 0.21...",1084
1,00dc31fe55e24d14989c89de4b3b683b,"[2017-01-20 17:00:01, 2017-01-20 17:00:02, 201...","[2017-01-20 17:00:01, 2017-01-20 17:00:02, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[19.927777777777777, 2.052777777777777, -0.216...",1344


#### ft_sum_hard_brakes_10_flg_val

In [0]:
def get_ft_sum_hard_brakes_10_flg_val(lst, datetimelist):  

  datetimelist_updated = [datetime.strptime(x,'%Y-%m-%d %H:%M:%S') for x in datetimelist]

  new_list = []  
  count = 0
  flag = 0
  i = 0
  nStartIndexPerBlock = 0
  nEndIndexPerBlock   = 0
  nDiffSumofTimes = 0

  n = len(lst)
  bValidBlock = False

  for i in range(0, len(lst)):
    if (i+1 < len(lst)):  
      if (lst[i] >= lst[i+1]):
        if(lst[i] > 0):
          bValidBlock = True

        if(flag != 1):
          nStartIndexPerBlock = i
        flag = 1
        new_list.append(0)
        continue
      elif (lst[i] < lst[i+1]):       
        if(flag == 1):
          if(lst[i] <= -10 and bValidBlock == True):          
            count = count+1
            new_list.append(1)
            nEndIndexPerBlock = i
            nDiffSumofTimes = nDiffSumofTimes + int((datetimelist_updated[nEndIndexPerBlock] - datetimelist_updated[nStartIndexPerBlock]).total_seconds())
            bValidBlock = False
          elif(bValidBlock == True): 
            bValidBlock = False
            new_list.append(0)
          else:
            new_list.append(0)
            flag = 0          
        else:
          new_list.append(0)
        continue
  if(flag == 1 and lst[n-1] <= -10 and bValidBlock == True):  
    new_list.append(1)
    count = count + 1  
    nDiffSumofTimes = nDiffSumofTimes + int((datetimelist_updated[n-1] - datetimelist_updated[nStartIndexPerBlock]).total_seconds())
    final_seconds = nDiffSumofTimes
  else:
    new_list.append(0)	
  return count

In [0]:
drive_df_trips_lst['ft_sum_hard_brakes_10_flg_val']  = drive_df_trips_lst.apply(lambda row: get_ft_sum_hard_brakes_10_flg_val(row['acceleration'], row['datetime']), axis=1)

In [70]:
drive_df_trips_lst.head(2)

,trip_id,datetime,datetime_1,time_diff_secs,acceleration,ft_cnt_vehicle_deaccel_val,ft_sum_hard_brakes_10_flg_val
0,00922df3be5a4589ab385d0c2da2dd81,"[2017-01-06 15:00:01, 2017-01-06 15:00:02, 201...","[2017-01-06 15:00:01, 2017-01-06 15:00:02, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[17.044444444444444, -3.8499999999999996, 0.21...",1084,15
1,00dc31fe55e24d14989c89de4b3b683b,"[2017-01-20 17:00:01, 2017-01-20 17:00:02, 201...","[2017-01-20 17:00:01, 2017-01-20 17:00:02, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[19.927777777777777, 2.052777777777777, -0.216...",1344,52


#### ft_sum_hard_brakes_3_flg_val

In [0]:
def get_ft_sum_hard_brakes_3_flg_val(lst, datetimelist):  

  datetimelist_updated = [datetime.strptime(x,'%Y-%m-%d %H:%M:%S') for x in datetimelist]

  new_list = []  
  count = 0
  flag = 0
  i = 0
  nStartIndexPerBlock = 0
  nEndIndexPerBlock   = 0
  nDiffSumofTimes = 0

  n = len(lst)
  bValidBlock = False

  for i in range(0, len(lst)):
    if (i+1 < len(lst)):  
      if (lst[i] >= lst[i+1]):
        if(lst[i] > 0):
          bValidBlock = True

        if(flag != 1):
          nStartIndexPerBlock = i
        flag = 1
        new_list.append(0)
        continue
      elif (lst[i] < lst[i+1]):       
        if(flag == 1):
          if(lst[i] <= -3 and lst[i] >= -10 and bValidBlock == True):          
            count = count+1
            new_list.append(1)
            nEndIndexPerBlock = i
            nDiffSumofTimes = nDiffSumofTimes + int((datetimelist_updated[nEndIndexPerBlock] - datetimelist_updated[nStartIndexPerBlock]).total_seconds())
            bValidBlock = False
          elif(bValidBlock == True): 
            bValidBlock = False
            new_list.append(0)
          else:
            new_list.append(0)
            flag = 0          
        else:
          new_list.append(0)
        continue
  if(flag == 1 and lst[n-1] <= -3 and lst[n-1] > -10 and bValidBlock == True):  
    new_list.append(1)
    count = count + 1  
    nDiffSumofTimes = nDiffSumofTimes + int((datetimelist_updated[n-1] - datetimelist_updated[nStartIndexPerBlock]).total_seconds())
    final_seconds = nDiffSumofTimes
  else:
    new_list.append(0)	
  return count

In [0]:
drive_df_trips_lst['ft_sum_hard_brakes_3_flg_val']  = drive_df_trips_lst.apply(lambda row: get_ft_sum_hard_brakes_3_flg_val(row['acceleration'], row['datetime']), axis=1)

In [73]:
drive_df_trips_lst.head(2)

,trip_id,datetime,datetime_1,time_diff_secs,acceleration,ft_cnt_vehicle_deaccel_val,ft_sum_hard_brakes_10_flg_val,ft_sum_hard_brakes_3_flg_val
0,00922df3be5a4589ab385d0c2da2dd81,"[2017-01-06 15:00:01, 2017-01-06 15:00:02, 201...","[2017-01-06 15:00:01, 2017-01-06 15:00:02, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[17.044444444444444, -3.8499999999999996, 0.21...",1084,15,521
1,00dc31fe55e24d14989c89de4b3b683b,"[2017-01-20 17:00:01, 2017-01-20 17:00:02, 201...","[2017-01-20 17:00:01, 2017-01-20 17:00:02, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[19.927777777777777, 2.052777777777777, -0.216...",1344,52,820


#### ft_sum_time_deaccel_val

In [0]:
def get_ft_cnt_vehicle_deaccel_val_summ(lst, datetimelist):
  
  datetimelist_updated = [datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S') for x in datetimelist]

  new_list = []  
  count = 0
  flag = 0
  i = 0
  nStartIndexPerBlock = 0
  nEndIndexPerBlock   = 0

  nDiffSumofTimes = 0
  

  n = len(datetimelist)

  for i in range(0, len(lst)):
    if (i+1 < len(lst)):  
      if (lst[i] >= lst[i+1]):
        if(flag != 1):
          nStartIndexPerBlock = i
        flag = 1
        new_list.append(0)
        continue
      elif (lst[i] < lst[i+1]):       
        if(flag == 1):
          count = count+1
          new_list.append(1)
          nEndIndexPerBlock = i
          nDiffSumofTimes = nDiffSumofTimes + int((datetimelist_updated[nEndIndexPerBlock] - datetimelist_updated[nStartIndexPerBlock]).total_seconds())
          flag = 0
        else:        
          new_list.append(0)
        continue
  if(flag == 1):  
    new_list.append(1)
    count = count + 1
    nDiffSumofTimes = nDiffSumofTimes + int((datetimelist_updated[n-1] - datetimelist_updated[nStartIndexPerBlock]).total_seconds())
    final_seconds = nDiffSumofTimes 
  else:
    new_list.append(0)
  return nDiffSumofTimes
	

In [0]:
drive_df_trips_lst['ft_sum_time_deaccel_val']  = drive_df_trips_lst.apply(lambda row: get_ft_cnt_vehicle_deaccel_val_summ(row['acceleration'], row['datetime']), axis=1)

In [76]:
drive_df_trips_lst.head(2)

,trip_id,datetime,datetime_1,time_diff_secs,acceleration,ft_cnt_vehicle_deaccel_val,ft_sum_hard_brakes_10_flg_val,ft_sum_hard_brakes_3_flg_val,ft_sum_time_deaccel_val
0,00922df3be5a4589ab385d0c2da2dd81,"[2017-01-06 15:00:01, 2017-01-06 15:00:02, 201...","[2017-01-06 15:00:01, 2017-01-06 15:00:02, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[17.044444444444444, -3.8499999999999996, 0.21...",1084,15,521,1456
1,00dc31fe55e24d14989c89de4b3b683b,"[2017-01-20 17:00:01, 2017-01-20 17:00:02, 201...","[2017-01-20 17:00:01, 2017-01-20 17:00:02, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[19.927777777777777, 2.052777777777777, -0.216...",1344,52,820,1838


#### ft_cnt_vehicle_accel_val

In [0]:
def get_ft_cnt_vehicle_accel_val(lst, datetimelist):  

  datetimelist_updated = [datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S') for x in datetimelist]

  new_list = []  
  count = 0
  flag = 0
  i = 0
  nStartIndexPerBlock = 0
  nEndIndexPerBlock   = 0
  nDiffSumofTimes = 0
  n = len(lst)

  for i in range(0, len(lst)):
    if (i+1 < len(lst)):  
      if (lst[i] >= lst[i+1]):
        if(flag != 1):
          nStartIndexPerBlock = i
        flag = 1
        new_list.append(0)
        continue
      elif (lst[i] < lst[i+1]):       
        if(flag == 1):
          count = count+1
          new_list.append(1)
          nEndIndexPerBlock = i
          nDiffSumofTimes = nDiffSumofTimes + int((datetimelist_updated[nEndIndexPerBlock] - datetimelist_updated[nStartIndexPerBlock]).total_seconds())
          flag = 0
        else:        
          new_list.append(0)
        continue
  if(flag == 1):  
    new_list.append(1)
    count = count + 1

    nDiffSumofTimes = nDiffSumofTimes + int((datetimelist_updated[n-1] - datetimelist_updated[nStartIndexPerBlock]).total_seconds())
    final_seconds = nDiffSumofTimes 
  else:
    new_list.append(0)
  return count
	

In [0]:
drive_df_trips_lst['ft_cnt_vehicle_accel_val']  = drive_df_trips_lst.apply(lambda row: get_ft_cnt_vehicle_accel_val(row['acceleration'], row['datetime']), axis=1)

In [79]:
drive_df_trips_lst.head(3)

,trip_id,datetime,datetime_1,time_diff_secs,acceleration,ft_cnt_vehicle_deaccel_val,ft_sum_hard_brakes_10_flg_val,ft_sum_hard_brakes_3_flg_val,ft_sum_time_deaccel_val,ft_cnt_vehicle_accel_val
0,00922df3be5a4589ab385d0c2da2dd81,"[2017-01-06 15:00:01, 2017-01-06 15:00:02, 201...","[2017-01-06 15:00:01, 2017-01-06 15:00:02, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[17.044444444444444, -3.8499999999999996, 0.21...",1084,15,521,1456,1084
1,00dc31fe55e24d14989c89de4b3b683b,"[2017-01-20 17:00:01, 2017-01-20 17:00:02, 201...","[2017-01-20 17:00:01, 2017-01-20 17:00:02, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[19.927777777777777, 2.052777777777777, -0.216...",1344,52,820,1838,1344
2,0156d21e316d4d8b9d5bf6ccff797bf7,"[2017-01-01 02:00:01, 2017-01-01 02:00:02, 201...","[2017-01-01 02:00:01, 2017-01-01 02:00:02, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[16.927777777777777, 0.56388888888889, -3.0333...",1229,1,218,1725,1229


#### ft_sum_hard_accel_10_flg_val

In [0]:
def get_ft_sum_hard_accel_10_flg_val(lst, datetimelist):  


  datetimelist_updated = [datetime.strptime(x,'%Y-%m-%d %H:%M:%S') for x in datetimelist]

  new_list = []  
  count = 0
  flag = 0
  i = 0
  nStartIndexPerBlock = 0
  nEndIndexPerBlock   = 0
  nDiffSumofTimes = 0

  n = len(lst)
  bValidBlock = False

  for i in range(0, len(lst)):
    if (i+1 < len(lst)):  
      if (lst[i] <= lst[i+1]):
        if(lst[i] < 0):
          bValidBlock = True

        if(flag != 1):
          nStartIndexPerBlock = i
        flag = 1
        new_list.append(0)
        continue
      elif (lst[i] > lst[i+1]):       
        if(flag == 1):
          if(lst[i] >= 10 and bValidBlock == True):          
            count = count+1
            new_list.append(1)
            nEndIndexPerBlock = i
            nDiffSumofTimes = nDiffSumofTimes + int((datetimelist_updated[nEndIndexPerBlock] - datetimelist_updated[nStartIndexPerBlock]).total_seconds())
            bValidBlock = False
          elif(bValidBlock == True): 
            bValidBlock = False
            new_list.append(0)
          else:
            new_list.append(0)
            flag = 0          
        else:
          new_list.append(0)
        continue
  if(flag == 1 and lst[n-1] >= 10 and bValidBlock == True):  
    new_list.append(1)
    count = count + 1  
    nDiffSumofTimes = nDiffSumofTimes + int((datetimelist_updated[n-1] - datetimelist_updated[nStartIndexPerBlock]).total_seconds())
    final_seconds = nDiffSumofTimes
  else:
    new_list.append(0)	
  return count

In [0]:
drive_df_trips_lst['ft_sum_hard_accel_10_flg_val']  = drive_df_trips_lst.apply(lambda row: get_ft_sum_hard_accel_10_flg_val(row['acceleration'], row['datetime']), axis=1)

#### ft_sum_hard_accel_3_flg_val

In [0]:
def get_ft_sum_hard_accel_3_flg_val(lst, datetimelist): 


  datetimelist_updated = [datetime.strptime(x,'%Y-%m-%d %H:%M:%S') for x in datetimelist]

  new_list = []  
  count = 0
  flag = 0
  i = 0
  nStartIndexPerBlock = 0
  nEndIndexPerBlock   = 0
  nDiffSumofTimes = 0

  n = len(lst)
  bValidBlock = False

  for i in range(0, len(lst)):
    if (i+1 < len(lst)):  
      if (lst[i] <= lst[i+1]):
        if(lst[i] < 0):
          bValidBlock = True

        if(flag != 1):
          nStartIndexPerBlock = i
        flag = 1
        new_list.append(0)
        continue
      elif (lst[i] > lst[i+1]):       
        if(flag == 1):
          if(lst[i] >= 3 and lst[i] < 10 and bValidBlock == True):          
            count = count+1
            new_list.append(1)
            nEndIndexPerBlock = i
            nDiffSumofTimes = nDiffSumofTimes + int((datetimelist_updated[nEndIndexPerBlock] - datetimelist_updated[nStartIndexPerBlock]).total_seconds())
            bValidBlock = False
          elif(bValidBlock == True): 
            bValidBlock = False
            new_list.append(0)
          else:
            new_list.append(0)
            flag = 0          
        else:
          new_list.append(0)
        continue
  if(flag == 1 and lst[n-1] >= 3 and lst[n-1] < 10 and bValidBlock == True):  
    new_list.append(1)
    count = count + 1  
    nDiffSumofTimes = nDiffSumofTimes + int((datetimelist_updated[n-1] - datetimelist_updated[nStartIndexPerBlock]).total_seconds())
    final_seconds = nDiffSumofTimes
  else:
    new_list.append(0)	
  return count

In [0]:
drive_df_trips_lst['ft_sum_hard_accel_3_flg_val']  = drive_df_trips_lst.apply(lambda row: get_ft_sum_hard_accel_3_flg_val(row['acceleration'], row['datetime']), axis=1)

#### ft_sum_time_accel_val

In [0]:
def get_ft_cnt_vehicle_accel_val_summ(lst, datetimelist):
  
  datetimelist_updated = [datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S') for x in datetimelist]

  new_list = []  
  count = 0
  flag = 0
  i = 0
  nStartIndexPerBlock = 0
  nEndIndexPerBlock   = 0

  nDiffSumofTimes = 0
  

  n = len(datetimelist)

  for i in range(0, len(lst)):
    if (i+1 < len(lst)):  
      if (lst[i] <= lst[i+1]):
        if(flag != 1):
          nStartIndexPerBlock = i
        flag = 1
        new_list.append(0)
        continue
      elif (lst[i] > lst[i+1]):       
        if(flag == 1):
          count = count+1
          new_list.append(1)
          nEndIndexPerBlock = i
          nDiffSumofTimes = nDiffSumofTimes + int((datetimelist_updated[nEndIndexPerBlock] - datetimelist_updated[nStartIndexPerBlock]).total_seconds())
          flag = 0
        else:        
          new_list.append(0)
        continue
  if(flag == 1):  
    new_list.append(1)
    count = count + 1
    nDiffSumofTimes = nDiffSumofTimes + int((datetimelist_updated[n-1] - datetimelist_updated[nStartIndexPerBlock]).total_seconds())
  else:
    new_list.append(0)
  return nDiffSumofTimes
	

In [0]:
drive_df_trips_lst['ft_sum_time_accel_val']  = drive_df_trips_lst.apply(lambda row: get_ft_cnt_vehicle_accel_val_summ(row['acceleration'], row['datetime']), axis=1)

In [86]:
drive_df_trips_lst.head(3)

,trip_id,datetime,datetime_1,time_diff_secs,acceleration,ft_cnt_vehicle_deaccel_val,ft_sum_hard_brakes_10_flg_val,ft_sum_hard_brakes_3_flg_val,ft_sum_time_deaccel_val,ft_cnt_vehicle_accel_val,ft_sum_hard_accel_10_flg_val,ft_sum_hard_accel_3_flg_val,ft_sum_time_accel_val
0,00922df3be5a4589ab385d0c2da2dd81,"[2017-01-06 15:00:01, 2017-01-06 15:00:02, 201...","[2017-01-06 15:00:01, 2017-01-06 15:00:02, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[17.044444444444444, -3.8499999999999996, 0.21...",1084,15,521,1456,1084,7,502,1442
1,00dc31fe55e24d14989c89de4b3b683b,"[2017-01-20 17:00:01, 2017-01-20 17:00:02, 201...","[2017-01-20 17:00:01, 2017-01-20 17:00:02, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[19.927777777777777, 2.052777777777777, -0.216...",1344,52,820,1838,1344,81,786,1850
2,0156d21e316d4d8b9d5bf6ccff797bf7,"[2017-01-01 02:00:01, 2017-01-01 02:00:02, 201...","[2017-01-01 02:00:01, 2017-01-01 02:00:02, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[16.927777777777777, 0.56388888888889, -3.0333...",1229,1,218,1725,1229,0,229,1707


In [87]:
drive_df_trips_lst.shape

(1708, 13)

In [88]:
drive_df_trips_lst.columns

Index(['trip_id', 'datetime', 'datetime_1', 'time_diff_secs', 'acceleration',
       'ft_cnt_vehicle_deaccel_val', 'ft_sum_hard_brakes_10_flg_val',
       'ft_sum_hard_brakes_3_flg_val', 'ft_sum_time_deaccel_val',
       'ft_cnt_vehicle_accel_val', 'ft_sum_hard_accel_10_flg_val',
       'ft_sum_hard_accel_3_flg_val', 'ft_sum_time_accel_val'],
      dtype='object')

In [89]:
drive_df_trips_lst.head(3)

,trip_id,datetime,datetime_1,time_diff_secs,acceleration,ft_cnt_vehicle_deaccel_val,ft_sum_hard_brakes_10_flg_val,ft_sum_hard_brakes_3_flg_val,ft_sum_time_deaccel_val,ft_cnt_vehicle_accel_val,ft_sum_hard_accel_10_flg_val,ft_sum_hard_accel_3_flg_val,ft_sum_time_accel_val
0,00922df3be5a4589ab385d0c2da2dd81,"[2017-01-06 15:00:01, 2017-01-06 15:00:02, 201...","[2017-01-06 15:00:01, 2017-01-06 15:00:02, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[17.044444444444444, -3.8499999999999996, 0.21...",1084,15,521,1456,1084,7,502,1442
1,00dc31fe55e24d14989c89de4b3b683b,"[2017-01-20 17:00:01, 2017-01-20 17:00:02, 201...","[2017-01-20 17:00:01, 2017-01-20 17:00:02, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[19.927777777777777, 2.052777777777777, -0.216...",1344,52,820,1838,1344,81,786,1850
2,0156d21e316d4d8b9d5bf6ccff797bf7,"[2017-01-01 02:00:01, 2017-01-01 02:00:02, 201...","[2017-01-01 02:00:01, 2017-01-01 02:00:02, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[16.927777777777777, 0.56388888888889, -3.0333...",1229,1,218,1725,1229,0,229,1707


In [90]:
drive_df_trips_lst_1 = drive_df_trips_lst[['trip_id','ft_cnt_vehicle_deaccel_val','ft_sum_time_accel_val',
                                           'ft_sum_time_deaccel_val','ft_cnt_vehicle_accel_val','ft_sum_hard_brakes_10_flg_val',
                                          'ft_sum_hard_brakes_3_flg_val', 'ft_sum_hard_accel_10_flg_val', 'ft_sum_hard_accel_3_flg_val'
                                          ]]
drive_df_trips_lst_1.head()

,trip_id,ft_cnt_vehicle_deaccel_val,ft_sum_time_accel_val,ft_sum_time_deaccel_val,ft_cnt_vehicle_accel_val,ft_sum_hard_brakes_10_flg_val,ft_sum_hard_brakes_3_flg_val,ft_sum_hard_accel_10_flg_val,ft_sum_hard_accel_3_flg_val
0,00922df3be5a4589ab385d0c2da2dd81,1084,1442,1456,1084,15,521,7,502
1,00dc31fe55e24d14989c89de4b3b683b,1344,1850,1838,1344,52,820,81,786
2,0156d21e316d4d8b9d5bf6ccff797bf7,1229,1707,1725,1229,1,218,0,229
3,01b8a24510cd4e4684d67b96369286e0,233,301,328,233,12,137,9,139
4,01c2a70c25e5428bb33811ca5eb19270,2612,3579,3558,2612,1,679,0,674


In [91]:
drive_df_trips_lst_1.shape

(1708, 9)

In [0]:
drive_df_trips_lst_1 = drive_df_trips_lst_1.sort_values(['trip_id'], ascending=[True])

In [0]:
# Convert the datafrae into CSV file and store it
drive_df_trips_lst_1.to_csv('drive_features.csv',index= False)

In [0]:
# from datetime import datetime
# from pytz import timezone
# import pytz

# date_format='%Y-%m-%d %H:%M:%S'
# # date = datetime.now(tz=pytz.utc)
# # date = to_datetime('2017-01-01 07:00:00')
# date = datetime.strptime(str('2017-01-01 07:00:00'),'%Y-%m-%d %H:%M:%S')

# print('Current date & time is:', date.strftime(date_format))

# date = date.astimezone(timezone('US/Pacific'))

# print('Local date & time is  :', date.strftime(date_format))

In [0]:
# 